In [ ]:
import cv2
import os
import numpy as np
import pickle
subjects = ["", "Abhishek", "ABC","XYZ"]
def detect_face(img):
     gray = cv2.cvtColor(img, cv2.COLOR_BGR2GRAY)
     #choose your preference
     face_cascade = cv2.CascadeClassifier('XML/haarcascade_frontalface_alt.xml')#for accurate result but prepare time high 
     #face_cascade = cv2.CascadeClassifier('XML/lbpcascade_frontalface.xml')#for low accurate result but prepare time low 
     faces = face_cascade.detectMultiScale(gray, scaleFactor=1.2, minNeighbors=5);
     if (len(faces) == 0):
        return None, None
     (x, y, w, h) = faces[0]
     return gray[y:y+w, x:x+h], faces[0]
def prepare_training_data(data_folder_path):
    dirs = os.listdir(data_folder_path)
    faces = []
    labels = []
    for dir_name in dirs:
        if not dir_name.startswith("a"):
            continue; 
        label = int(dir_name.replace("a", ""))
        subject_dir_path = data_folder_path + "/" + dir_name
        subject_images_names = os.listdir(subject_dir_path)
        for image_name in subject_images_names:
            if image_name.startswith("."):
                continue;
            image_path = subject_dir_path + "/" + image_name
            image = cv2.imread(image_path)
            #cv2.imshow("Training on image...", cv2.resize(image, (400, 500)))#for showing process remove comment 
            cv2.waitKey(100)
            face, rect = detect_face(image)
            if face is not None:
                faces.append(face)
                labels.append(label)
            cv2.destroyAllWindows()
    cv2.waitKey(1)
    cv2.destroyAllWindows()
    return faces, labels
def train(): #for first time train
    faces, labels = prepare_training_data("training-data")
    with open("faces.txt", "wb") as fp:
        pickle.dump(faces, fp)
    with open("labels.txt", "wb") as fp:
        pickle.dump(labels, fp)
    return faces,labels
def alreadytrain():
    with open('faces.txt', 'rb') as f:
        faces = pickle.load(f)
    with open('labels.txt', 'rb') as f:
        labels = pickle.load(f)
    return faces,labels


print("Preparing data...")
if os.path.isfile("faces.txt") & os.path.isfile("faces.txt") :
    print("you already set!!!")
    faces,labels = alreadytrain()
else:
    print("We are creating data from your face")
    faces,labels = train()
print("Data prepared")
print("Total faces: ", len(faces))
print("Total labels: ", len(labels))
face_recognizer = cv2.face.LBPHFaceRecognizer_create()
face_recognizer.train(faces, np.array(labels))
def draw_rectangle(img, rect):
    (x, y, w, h) = rect
    cv2.rectangle(img, (x, y), (x+w, y+h), (0, 255, 0), 2) 
def draw_text(img, text, x, y):
    cv2.putText(img, text, (x, y), cv2.FONT_HERSHEY_PLAIN, 2, (0, 255, 0), 2)
def predict(test_img):
    img = test_img.copy()
    face, rect = detect_face(img)
    label, confidence = face_recognizer.predict(face)
    label_text = subjects[label]
    draw_rectangle(img, rect)
    draw_text(img, label_text, rect[0], rect[1]-5)
    return img
print("Predicting images...")

cam = cv2.VideoCapture(0)

cv2.namedWindow("Capture Image")



while True:
    ret,image = cam.read()
    #print(ret)
    cv2.imshow("Capture_Image", image)
    if not ret:
        break
    k = cv2.waitKey(1)

    if k%256 == 27:
        # ESC pressed
        print("Escape hit, closing...")
        break
    if k%256 == 32:
        # SPACE pressed
        img_name = "testimage.jpg"
        cv2.imwrite(img_name, image)
        print("{} captured`!".format(img_name))
        break
try:
    cam.release()
    cv2.destroyAllWindows()
    test_img0 = cv2.imread("testimage.jpg")
    predicted_img0 = predict(test_img0)

    print("Prediction complete")
    cv2.imshow(subjects[0], cv2.resize(predicted_img0, (400, 400)))
    cv2.waitKey(0)
    cv2.destroyAllWindows()
    #os.remove(testimage.jpg)
except:
    print("unknown")
